In [1]:
import csv
with open('result_ml_bulk.csv', 'r') as file:
    contents = csv.reader(file)
    
    data, flag, names, modeldict = [], True, [], {}
    Data = []
    count = 0
    for row in contents:
            if flag:
                for r in row: names.append(r)
                flag = False
            else:
                data = [float(row[0]), float(row[1]), int(row[2]),int(row[3]),int(row[4]),int(row[5]),int(row[6]),int(row[7]),int(row[8]),int(row[9]),int(row[10]),float(row[11]),float(row[12])]
                Data.append(data)

In [2]:
I, O = [], []
with open('result_ml_bulk.csv', 'r') as file:
    contents = csv.reader(file)
    flag = True
    for row in contents:
        if flag:
            for r in row: names.append(r)
            flag = False
        else:
            #data = [float(row[0]), float(row[1]), int(row[2]),int(row[3]),int(row[4]),int(row[5]),int(row[6]),int(row[7]),int(row[8]),int(row[9]),int(row[10]),float(row[11]),float(row[12])]
            fin_size = float(row[0])
            bc=float(row[1])
            bw=float(row[2])
            model = [float(x) for x in row[3:9]]
            uni=float(row[10])
            i = [fin_size,bc,bw]
            i = i + model
            i.append(uni)
            I.append(i)

            slc2 = row[-2]
            #slc6 = row[-1]
            O.append(slc2)#, slc6])

In [3]:
maximum = max([float(o) for o in O[1:]])
O[1:] = [float(o)/maximum for o in O[1:]]

In [4]:
import torch, torch.nn as nn, torch.nn.functional as F

class FeedNet(nn.Module):
    
    def __init__(self):
        super(FeedNet, self).__init__()
        self.layer1 = nn.Linear(10, 16)
        self.layer2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = self.layer2(out)
        out = self.sigmoid(out)
        return out

In [5]:
# # 80% train and 20% test

# In[5]:


import random
full = range(1, len(I)-1)
train = random.sample(full, int(0.8*len(I)))
test = list(set(full) - set(train))

In [6]:
# # train the model

# In[6]:

from tqdm import tqdm
criterion = nn.MSELoss()
I, O = torch.FloatTensor(I[1:]), torch.FloatTensor(O[1:]).unsqueeze(1)
FFNN = FeedNet()

optimiser = torch.optim.Adam(FFNN.parameters(), lr=0.001)
FFNN.train()
for _ in tqdm(range(500)):
    for i in train:
        optimiser.zero_grad()
        P = FFNN(I[i])
        loss = criterion(P, O[i])
        loss.backward()
        optimiser.step()

100%|██████████| 500/500 [01:53<00:00,  4.48it/s]


In [7]:

# # test the model

# In[14]:


FFNN.eval()
for i in test:
    P = FFNN(I[i])
    print("datapoint", i, ", actual:", float(O[i]*maximum), ", and predicted:", float(P*maximum))

datapoint 256 , actual: 1.1781227588653564 , and predicted: 1.4346486330032349
datapoint 129 , actual: 0.9002285003662109 , and predicted: 0.798508882522583
datapoint 257 , actual: 1.0649906396865845 , and predicted: 1.7662811279296875
datapoint 259 , actual: 0.52049320936203 , and predicted: 0.6539555191993713
datapoint 262 , actual: 1.0203921794891357 , and predicted: 1.380662441253662
datapoint 135 , actual: 0.36610594391822815 , and predicted: 1.1895500421524048
datapoint 136 , actual: 1.0590840578079224 , and predicted: 0.9707424640655518
datapoint 137 , actual: 0.4801871180534363 , and predicted: 1.2087163925170898
datapoint 266 , actual: 0.6320168972015381 , and predicted: 0.8101460933685303
datapoint 267 , actual: 1.5093810558319092 , and predicted: 1.4943362474441528
datapoint 269 , actual: 1.2390562295913696 , and predicted: 1.5527775287628174
datapoint 15 , actual: 1.0030618906021118 , and predicted: 0.43939223885536194
datapoint 17 , actual: 0.1673341989517212 , and predict

In [8]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }}
);


<IPython.core.display.Javascript object>